In [ ]:
import sys
import os
from dotenv import load_dotenv
load_dotenv()
ROOT_DIR_PATH = os.environ.get('ROOT_PATH')
sys.path.append(os.path.abspath(ROOT_DIR_PATH))  # Adds root directory to sys.path

In [ ]:
from utils.data_loader import DatasetLoader
from utils.config_loader import load_config
print('loading config')
# Load config
config = load_config(f"{ROOT_DIR_PATH}/config/vit_config.yaml")

# *************  choosing the DATASET & MODEL *************

dataset_config = config["data"]['CALTECH256']
trainingConfig = config['training_dummy']

# **********************************************************

# data
DATASET = dataset_config["dataset"]
DATA_DIR =f'{ROOT_DIR_PATH}/data/{DATASET}/'
BATCH = dataset_config["batch_size"]
NUM_WORKERS = dataset_config["num_workers"]
IMAGE = dataset_config["img_size"]
NUM_CLASSES = dataset_config["num_classes"]
CHANNELS = dataset_config["channels"]
if DATASET == 'TINYIMAGENET200':
    SUBSET_ENABLED = dataset_config['subset_enabled']
    SUBSET_SIZE = dataset_config['subset_size']

# loading data
print(f'loading dataset : {DATASET}')
loader = DatasetLoader(training_config=trainingConfig,
                        dataset_name=DATASET,
                        data_dir=DATA_DIR,
                        batch_size=BATCH,
                        num_workers=NUM_WORKERS,
                        img_size=IMAGE)
train_loader, val_loader = loader.get_loaders()
print(f"Train batches: {len(train_loader)}, Validation batches: {len(val_loader)}")
print('data sanity check')
for images, labels in train_loader:
    print(f'image shape and labels shape in training data - one batch : {images.shape}, {labels.shape}')
    break

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, datasets
from timm.models.vision_transformer import vit_tiny_patch16_224
import os
import gc
from collections import defaultdict

torch.cuda.empty_cache()
torch.cuda.ipc_collect()
gc.collect()

from timm.models.vision_transformer import vit_tiny_patch16_224

BATCH_SIZE = 32
NUM_CLASSES = 10
EPOCHS = 25
LR = 0.001
IMG_SIZE = 224

# ========== Model ==========
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = vit_tiny_patch16_224(pretrained=False, num_classes=NUM_CLASSES).to(DEVICE)

# ========== Loss & Optimizer ==========
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# ========== Train Loop ==========
for epoch in range(EPOCHS):
    model.train()
    total, correct = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = 100. * correct / total
    print(f"Epoch {epoch+1:02d} | Train Acc: {acc:.2f}%")

    # stop early if we hit 100% train acc
    if acc == 100.0:
        print("Model has successfully overfitted the subset.")
        break


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, datasets
from timm.models.vision_transformer import vit_tiny_patch16_224
import os
import gc
from collections import defaultdict

torch.cuda.empty_cache()
torch.cuda.ipc_collect()
gc.collect()

BATCH_SIZE = 32
NUM_CLASSES = 10
EPOCHS = 25
LR = 0.001
IMG_SIZE = 224

# ========== Model ==========
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class TwoLayerCNN(nn.Module):
    def __init__(self, num_classes, img_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # (B, 32, 64, 64)
            nn.ReLU(),
            nn.MaxPool2d(2),  # (B, 32, 32, 32)
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # (B, 64, 32, 32)
            nn.ReLU(),
            nn.MaxPool2d(2),  # (B, 64, 16, 16)
            nn.Flatten(),  # (B, 64*16*16)
            nn.Linear(64 * (img_size // 4) * (img_size // 4), 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)
model = TwoLayerCNN(num_classes=NUM_CLASSES, img_size=IMG_SIZE).to(DEVICE)

# ========== Loss & Optimizer ==========
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# ========== Train Loop ==========
for epoch in range(EPOCHS):
    model.train()
    total, correct = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = 100. * correct / total
    print(f"Epoch {epoch+1:02d} | Train Acc: {acc:.2f}%")

    # stop early if we hit 100% train acc
    if acc == 100.0:
        print("Model has successfully overfitted the subset.")
        break


# Mean / STD DEV

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# Path to your Caltech256 training set (change if needed)
data_path = "/home/wd/Documents/work_stuff/ViT_REPLICATION/data/CALTECH256/train"

# Resize and convert to tensor (no normalization yet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a fixed size
    transforms.ToTensor()
])

# Load dataset
dataset = datasets.ImageFolder(root=data_path, transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2)

# Initialize sums
mean = 0.
std = 0.
nb_samples = 0.

# Accumulate mean and std across batches
for data, _ in tqdm(loader, desc="Computing mean/std"):
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)  # [B, C, H*W]
    mean += data.mean(2).sum(0)  # Sum of mean across batch
    std += data.std(2).sum(0)    # Sum of std across batch
    nb_samples += batch_samples

# Final average
mean /= nb_samples
std /= nb_samples

print(f"\nDataset Mean: {mean.tolist()}")
print(f"Dataset Std : {std.tolist()}")

# Loading balanced Subset

In [1]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()
ROOT_DIR_PATH = os.environ.get('ROOT_PATH')
sys.path.append(os.path.abspath(ROOT_DIR_PATH))  # Adds root directory to sys.path

from utils.config_loader import load_config
from utils.data_loader import DatasetLoader
import torch
from model.vit_custom import VisionTransformerTiny
from torch.amp import GradScaler
from torch.amp import autocast
from transformers import get_cosine_schedule_with_warmup
from timm.data import Mixup
import time
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np

/home/wd/Documents/work_stuff/ViT_REPLICATION/_vit_rep_py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = load_config(f"{ROOT_DIR_PATH}/config/vit_config.yaml")
dataset_config = config["data"]['CALTECH256']
specific_config = config["model"]['VIT_TINYV3']
trainingConfig = config['training_dummy']
DATASET = dataset_config["dataset"]
DATA_DIR =f'{ROOT_DIR_PATH}/data/{DATASET}/'
NUM_WORKERS = dataset_config["num_workers"]

In [3]:
# loading data
print(f'loading dataset : {DATASET}')
loader = DatasetLoader(training_config=trainingConfig,
                        dataset_name=DATASET,
                        data_dir=DATA_DIR,
                        num_workers=NUM_WORKERS)
train_loader, val_loader = loader.get_loaders()
print(f"Train batches: {len(train_loader)}, Validation batches: {len(val_loader)}")
print('data sanity check')
for images, labels in train_loader:
    print(f'image shape and labels shape in training data - one batch : {images.shape}, {labels.shape}')
    break

loading dataset : CALTECH256
Dataset directory /home/wd/Documents/work_stuff/ViT_REPLICATION/data/CALTECH256 already exists, zip downloaded.
getting balanced subset - class count : 10 - sample per class : 60

[TRAINING] Selected Classes and Sample Counts:

Class '010.beer-mug' (original label=9): total images = 75, using = 60
Class '020.brain-101' (original label=19): total images = 66, using = 60
Class '027.calculator' (original label=26): total images = 80, using = 60
Class '173.rifle' (original label=172): total images = 84, using = 60
Class '174.rotary-phone' (original label=173): total images = 67, using = 60
Class '176.saddle' (original label=175): total images = 88, using = 60
Class '177.saturn' (original label=176): total images = 76, using = 60
Class '191.sneaker' (original label=190): total images = 88, using = 60
Class '211.tambourine' (original label=210): total images = 76, using = 60
Class '254.greyhound' (original label=253): total images = 76, using = 60

Total selected

In [4]:
# loading model
modelConfigDict = {
    'CHANNEL' : dataset_config["channels"],
    'PATCH' : specific_config['patch_size'],
    'EMBEDDING' : specific_config['emb_size'],
    'IMAGE' : trainingConfig['image_size'],
    'NUM_HEADS' : specific_config['num_heads'],
    'MLP_RATIO' : specific_config['mlp_ratio'],
    'DROPOUT' : specific_config['dropout'],
    'NUM_CLASSES' : trainingConfig['num_subset_class'] if trainingConfig['apply_class_balance'] else dataset_config['num_classes'],
    'DEPTH' : specific_config['depth'],
    'QKV_BIAS':specific_config['qkv_bias'],
    'ATTN_DROP_RATE': specific_config['attn_drop_rate'],
    'DROP_PATH_RATE': specific_config['drop_path_rate']
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VisionTransformerTiny(**modelConfigDict).to(device)
print('Data Configuration:')
for k, v in dataset_config.items():
    print(f"  {k}: {v}")
print("Training Configuration:")
for k, v in trainingConfig.items():
    print(f"  {k}: {v}")


********* Model Params *********
{'CHANNEL': 3, 'PATCH': 16, 'EMBEDDING': 192, 'IMAGE': 224, 'NUM_HEADS': 3, 'MLP_RATIO': 4.0, 'DROPOUT': 0.0, 'NUM_CLASSES': 10, 'DEPTH': 8, 'QKV_BIAS': True, 'ATTN_DROP_RATE': 0.0, 'DROP_PATH_RATE': 0.1, '__class__': <class 'model.vit_custom.VisionTransformerTiny'>}


Data Configuration:
  dataset: CALTECH256
  data_path: /home/wd/Documents/work_stuff/ViT_REPLICATION/data/CALTECH256
  channels: 3
  num_workers: 8
  mean_aug: [0.5531906485557556, 0.5342377424240112, 0.5071621537208557]
  std_aug: [0.23687367141246796, 0.2358572781085968, 0.2385127693414688]
  num_classes: 256
Training Configuration:
  mixup: {'enabled': True, 'mixup_alpha': 0.1, 'cutmix_alpha': 0.4, 'label_smoothing_mixup': 0.2}
  epochs: 200
  lr: 0.0003
  weight_decay: 0.01
  scheduler_warmup: True
  warmup_steps: 30
  label_smoothing_enabled: False
  label_smoothing: 0.0
  es_patience: 30
  es_improv_delta: 0.001
  augmentation_enabled: True
  apply_class_balance: True
  num_subset_

In [5]:
### training/validation

def train_one_epoch(model, loader, criterion, optimizer, scaler, device,
                    mixup_fn=None, scheduler_warmup_enabled=False, scheduler_warmup=None):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    progress_bar = tqdm(loader, desc="Training", leave=True)
    for  inputs, targets in progress_bar:
        #print(f'input shape : {inputs.shape}, taget_shape : {targets.shape}, target dim : {targets.ndim}')
        inputs, targets = inputs.to(device), targets.to(device)
        if mixup_fn is not None:
            inputs, targets = mixup_fn(inputs, targets)

        if targets.ndim == 2:
            targets = targets.type_as(inputs)

        optimizer.zero_grad()
        with autocast(device_type='cuda'):
            outputs = model(inputs)
            loss = criterion(outputs, targets)
        scaler.scale(loss).backward()  
        scaler.step(optimizer)         
        scaler.update()                

        if scheduler_warmup_enabled:
            scheduler_warmup.step()

        running_loss += loss.item() * inputs.size(0)

        if targets.ndim == 2:
            # MixUp with soft labels
            _, predicted = outputs.max(1)
            _, true_classes = targets.max(1)  # Take argmax of soft labels as true class
            correct += predicted.eq(true_classes).sum().item()
            total += targets.size(0)
        else :
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)

        # Update progress bar with metrics
        if total > 0:
            avg_loss = running_loss / total
            accuracy = 100. * correct / total
            progress_bar.set_postfix({
                "Loss": f"{avg_loss:.4f}",
                "Acc": f"{accuracy:.2f}%"
            })

        else : raise Exception(f'Expected non-zero batch size, but got 0 targets. Check if the dataset is empty or DataLoader is misconfigured.')

    
    return avg_loss, accuracy

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    top5_correct = 0
    total = 0
    progress_bar = tqdm(loader, desc="Validation", leave=True)
    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            with autocast(device_type='cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            # Compute accuracy
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            # Top-5 Accuracy
            _, top5_preds = outputs.topk(5, dim=1, largest=True, sorted=True)
            top5_correct += sum([labels[i] in top5_preds[i] for i in range(len(labels))])

            total += labels.size(0)

            avg_loss = running_loss / total
            accuracy = 100. * correct / total
            top5_accuracy = 100. * top5_correct / total

            progress_bar.set_postfix({
                "Loss": f"{avg_loss:.4f}",
                "Acc": f"{accuracy:.2f}%"
            })
                
    return avg_loss, accuracy, top5_accuracy



In [6]:
### augmentations and improvements
mixupConfig = trainingConfig['mixup']
MIXUP_ALPHA = mixupConfig["mixup_alpha"]
CUTMIX_ALPHA = mixupConfig["cutmix_alpha"]
LABEL_SMOOTHENING_MIXUP = mixupConfig["label_smoothing_mixup"]
USE_MIXUP = mixupConfig["enabled"]

LEARNING_RATE = trainingConfig['lr']
EPOCHS = trainingConfig['epochs']
WEIGHT_DECAY = trainingConfig['weight_decay']
USE_SCHEDULER_WARMUP = trainingConfig['scheduler_warmup']
WARMUP_STEPS = trainingConfig['warmup_steps']
USE_LABEL_SMOOTHENING = trainingConfig["label_smoothing_enabled"]
LABEL_SMOOTHENING = trainingConfig["label_smoothing"]
EARLY_STOPPING_PATIENCE = trainingConfig["es_patience"]
EARLY_STOPPING_IMPROVEMENT_DELTA = trainingConfig["es_improv_delta"]

# === Mixup Setup ===
mixup_fn = None
if USE_MIXUP:
    mixup_fn = Mixup(
        mixup_alpha=MIXUP_ALPHA,
        cutmix_alpha=CUTMIX_ALPHA,
        label_smoothing=LABEL_SMOOTHENING_MIXUP,
        num_classes=trainingConfig['num_subset_class'] if trainingConfig['apply_class_balance'] else dataset_config['num_classes']
    )
    train_criterion = nn.BCEWithLogitsLoss()
    val_criterion = nn.CrossEntropyLoss(label_smoothing=0.0) #NO LABEL SMOOTHENING during validation
else:        
    mixup_fn = None
    train_criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHENING if USE_LABEL_SMOOTHENING else 0.0)
    val_criterion =  nn.CrossEntropyLoss(label_smoothing=0.0) #NO LABEL SMOOTHENING during validation

# optimizer & scheduler
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler_warmup_obj = None
scheduler_warmup_enabled_flag = False
if USE_SCHEDULER_WARMUP:
    scheduler_warmup_enabled_flag = True
    num_training_steps = EPOCHS * len(train_loader)
    num_warmup_steps = WARMUP_STEPS * len(train_loader)

    scheduler_warmup = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )
    scheduler_warmup_obj = scheduler_warmup

# scaler
scaler = GradScaler() 

In [7]:
startTime = time.time()
# monitors initialization
best_val_acc = 0.0
best_val_loss = np.inf

for epoch in range(1,EPOCHS+1):
    print(f"\nEpoch {epoch}/{EPOCHS}")

    train_loss, train_acc = train_one_epoch(model, train_loader, train_criterion, optimizer, scaler, device,
                                            mixup_fn=mixup_fn,
                                            scheduler_warmup_enabled=scheduler_warmup_enabled_flag,
                                            scheduler_warmup=scheduler_warmup_obj)
    val_loss, val_acc, val_acc_top5 = validate(model, val_loader, val_criterion, device)

    # monitoring learning rate variation
    current_lr = optimizer.param_groups[0]['lr']
    
    import copy
    if val_acc > best_val_acc + EARLY_STOPPING_IMPROVEMENT_DELTA:
        best_val_acc = val_acc
        best_val_acc_top5 = val_acc_top5
        best_val_loss = val_loss
        corresponding_train_acc = train_acc
        corresponding_train_loss = train_loss
        best_epoch = epoch
        best_model_state = copy.deepcopy(model.state_dict())
        best_optimizer_state = copy.deepcopy(optimizer.state_dict())
        if USE_SCHEDULER_WARMUP : best_scheduler_state = copy.deepcopy(scheduler_warmup_obj.state_dict())
        else : best_scheduler_state = None
        best_scaler_state = copy.deepcopy(scaler.state_dict())

        epochs_without_improvement = 0
    else:
        epochs_without_improvement+=1

    # Early stopping condition
    if epochs_without_improvement >= EARLY_STOPPING_PATIENCE:
        print(f"\nEarly stopping triggered. No improvement(delta={EARLY_STOPPING_IMPROVEMENT_DELTA}) in val_acc for {EARLY_STOPPING_PATIENCE} consecutive epochs.")
        break

    endTime = time.time()
    elapsedTime = endTime - startTime
    hours = int(elapsedTime // 3600)
    minutes = int((elapsedTime % 3600) // 60)
    seconds = int(elapsedTime % 60)
    print(f"Elapsed Time : {hours}h : {minutes}m : {seconds}s")

# saving the best state
print('\n====== Model Performance =======')
print(f"Train     Loss: {corresponding_train_loss:.4f},    Accuracy: {corresponding_train_acc:.2f}%")
print(f"Val(Best) Loss: {best_val_loss:.4f}, Accuracy: {best_val_acc:.2f}%, Top5 Accuracy: {best_val_acc_top5:.2f}%")

print(f"average time per epoch: {int((elapsedTime/epoch)/60)}")
print('\n\n-----------')
print(f"Best model Epoch : {best_epoch}")
print('-----------\n')
print(f"\nTraining completed in: {hours}h : {minutes}m : {seconds}s\n\n")



Epoch 1/200


Validation: 100%|██████████| 4/4 [00:00<00:00,  7.44it/s, Loss=2.2988, Acc=12.12%]


Elapsed Time : 0h : 0m : 2s

Epoch 2/200


Validation: 100%|██████████| 4/4 [00:00<00:00, 12.34it/s, Loss=2.2690, Acc=15.15%]


Elapsed Time : 0h : 0m : 5s

Epoch 3/200


...

Elapsed Time : 0h : 3m : 45s

Epoch 102/200


Validation: 100%|██████████| 4/4 [00:00<00:00, 11.97it/s, Loss=1.7126, Acc=38.89%]


Elapsed Time : 0h : 3m : 47s

Epoch 103/200


Validation: 100%|██████████| 4/4 [00:00<00:00, 11.86it/s, Loss=1.7214, Acc=38.89%]


Early stopping triggered. No improvement(delta=0.001) in val_acc for 30 consecutive epochs.

====== Model Performance =======
Train     Loss: 0.2897,    Accuracy: 39.00%
Val(Best) Loss: 1.7732, Accuracy: 40.40%, Top5 Accuracy: 87.37%
average time per epoch: 0


-----------
Best model Epoch : 73
-----------


Training completed in: 0h : 3m : 47s




In [ ]:
()